### Obligatory imports

In [ ]:
import pandas as pd
import time
import datetime as dt

### Download from the newest set of facilities from HFR and dump in the data folder. Read it with pandas.

In [ ]:
#Database of all the facility IDs, concatenate the variable value to URL.
#Click Excel Export:
#https://hfr-portal.ucchosting.co.tz/index.php?r=facilities/facilitiesWithNoGeo #no funciona
#https://hfr-portal.ucchosting.co.tz/index.php?r=facilities/facilitiesWithGeo #no funciona
#http://moh.go.tz/hfrportal/index.php?r=facilities/facilitiesWithNoGeo
#http://moh.go.tz/hfrportal/index.php?r=facilities/facilitiesWithGeo

url = 'http://moh.go.tz/hfrportal/index.php?r=facilities/view&facility_id='
date = dt.datetime.now().strftime("%Y-%m-%d")

In [ ]:
#This might raise an XLRDError Unsupported Format. Recommendation is to open the file and resave it with .xls extension
#Might potentially be readable as HTML
noGeo = pd.read_excel('./data/Facility_List_noGeo.xls', header = 2)
withGeo = pd.read_excel('./data/Facility_List_withGeo.xls', header = 2)

In [ ]:
display(noGeo.head())
display(withGeo.head())

In [ ]:
db = pd.concat([noGeo, withGeo])

In [ ]:
db.head()
db.shape

## Run the thing - takes approx. 15-17 hours

In [ ]:
#Using the IDs from the download, scrape all the websites.

transposed = []
failed = []
print('Running through the facility numbers now...')
counter = 0
for e in db['Facility Number']: #e is ID, ~10310 ids total
    if counter > 0:
        try:
            #When adding more rows, we shouldn't concatenate with the headers every time.
            # We need to treat subsequent dfs, slightly differently.
            time.sleep(3)
            table2 = pd.read_html(url+e) #(url+ID)
            twoTable = pd.concat(table2, ignore_index=True) #Every dataframe, must have an ID column added to it.
            twoTable.loc[-1] = 'Facility_ID', e #Assign the ID to the last row
            transposed.append(twoTable.transpose().drop([0])) #Add this to a list to be concat'd, doesn't have header
            time.sleep(1)
            counter += 1
            
            if counter % 100 == 0:
                print("Have done:",counter)
        
        except Exception as i:
            print(i)
            failed.append(url+e)
            time.sleep(120)
            continue
    else:
        table = pd.read_html(url+e) #(url+ID)
        oneTable = pd.concat(table, ignore_index=True) #Every dataframe, must have an ID column added to it.
        oneTable.loc[-1] = 'Facility_ID', e #Assign the ID to the last row
        transposed.append(oneTable.transpose()) #Add this to a list to be concat'd
        counter += 1

print("Done scraping.")
fullFacility = pd.concat(transposed, ignore_index=True)
facility_columns = fullFacility.loc[0, :].tolist()

for e in facility_columns:
    facility_columns[facility_columns.index(e)] = e.replace(" ", "_")
    e = e.replace(" ", "_")
    facility_columns[facility_columns.index(e)] = e.replace("'s", "")

fullFacility.columns = facility_columns
fullFacility.drop([0], inplace=True)
fullFacility.reset_index(drop=True)
ordered_columns = [facility_columns[-1]]+facility_columns[:-1]
fullFacility[ordered_columns].to_csv('./data/Health_Facilities_'+date+'.csv', index=False)

In [ ]:
#TODO: Use list in transposed to list every df's df.columns
#See what the column setup is like

In [ ]:
failed

### IF ANY FAIL, USE THIS

In [ ]:
#If any fail, use this:
import re

transpose_fail = []
for fail in failed:    
    tables = pd.read_html(fail) #(url+ID) has multiple tables on page that we need to pull
    Table = pd.concat(tables, ignore_index=True) #Every dataframe, must have an ID column added to it.
    e = re.findall("[0-9\-]+", fail)[0]
    Table.loc[-1] = 'Facility_ID', e #Assign the ID to the last row
    transpose_fail.append(Table.transpose().drop([0])) #Add this to a list to be concat'd, doesn't have header

fullFacility = pd.concat(transposed+transpose_fail, ignore_index=True)
facility_columns = fullFacility.loc[0, :].tolist()

for e in facility_columns:
    facility_columns[facility_columns.index(e)] = e.replace(" ", "_")
    e = e.replace(" ", "_")
    facility_columns[facility_columns.index(e)] = e.replace("'s", "")

fullFacility.columns = facility_columns
fullFacility.drop([0], inplace=True)
fullFacility.reset_index(drop=True)
ordered_columns = [facility_columns[-1]]+facility_columns[:-1]
fullFacility[ordered_columns].to_csv('./data/Health_Facilities_'+date+'.csv', index=False)

### Merge the original and scraped dataframes for a complete dataset

In [ ]:
#It's merging time!
merge_df = db[['Facility Number', 'Facility Name', 'Ward', 'Village/Street', 'Facility Type', 'Operating Status', 'Ownership', 
 'Latitude', 'Longitude']] #Taking subset of original dataframe columns that are not duplicates

#Match ID column name for merging
merge_columns = merge_df.columns.tolist()
merge_columns[0] = 'Facility_ID'
merge_df.columns = merge_columns

combined_data = pd.merge(merge_df, fullFacility[ordered_columns], how='inner', on='Facility_ID')
del combined_data['Geo-coordinates(Latitude,Longitude)'] #Remove duplicate and wrongly formatted coordinate column
combined_data.to_csv('./data/Health_Facilities_'+date+'.csv', index=False)